#### Load and prepare data:

In [1]:
import plotly.plotly as py
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool, NumeralTickFormatter
from bokeh.layouts import gridplot
from bokeh.charts import Bar, output_file, Scatter, BoxPlot
from bokeh.palettes import Spectral6

# Import necessary module
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# Create engine: engine
engine = create_engine('sqlite:///programming.sqlite')

In [3]:
df = pd.read_sql_query("SELECT distinct Disciplina from Grade", engine)

#df

## Análise 1 - Situação dos alunos que foram bem em ITP

In [4]:
itp_notas = {}
edb1_notas = {}

def retrieve_edb1_info(aid):
    tmp = pd.read_sql_query('''
            SELECT Aluno_ID, [Média Final]
            FROM Grade
            WHERE Aluno_ID = ''' + str(aid) + '''
                AND Disciplina = "IMD0029 - ESTRUTURA DE DADOS BÁSICAS I"
            LIMIT 1;''', engine
    )
    if tmp.empty != True:
        try:
            media_edb1 = float(tmp['Média Final'])
            return media_edb1
        except TypeError:
            return 0.0
    return None

itp_best = pd.read_sql_query('''SELECT Aluno_ID, [Média Final]
                       FROM Grade
                       WHERE [Média Final] >= 7.0
                           AND Disciplina = "IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO"''',
                       engine)

for aid, med in zip(itp_best['Aluno_ID'], itp_best['Média Final']):
    edb1 = retrieve_edb1_info(aid)
    if edb1 != None:
        itp_notas[int(aid)] = float(med)
        edb1_notas[int(aid)] = float(edb1)

In [5]:
red_itp = []
red_edb1 = []
green_itp = []
green_edb1 = []
blue_itp = []
blue_edb1 = []

for x in sorted(itp_notas.keys()):
    if edb1_notas[int(x)] < 5.0:
        red_itp.append(itp_notas[x])
        red_edb1.append(edb1_notas[int(x)])
    elif edb1_notas[int(x)] >= 5.0 and edb1_notas[int(x)] < 7.0:
        green_itp.append(itp_notas[x])
        green_edb1.append(edb1_notas[int(x)])
    elif edb1_notas[int(x)] >= 7.0:
        blue_itp.append(itp_notas[x])
        blue_edb1.append(edb1_notas[int(x)])

p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Média em EDB1', y_axis_label = 'Média em ITP',
           title = 'Desempenho dos alunos que cursaram ITP e EDB1 em seguida')

p.circle(x = red_edb1, y = red_itp, color = "red", size = 7)
p.circle(x = green_edb1, y = green_itp, color = "green", size = 7)
p.circle(x = blue_edb1, y = blue_itp, color = "blue", size = 7)

GlyphRenderer(id='0eea148a-aacd-47e4-93a0-c16cb09a3ac7', ...)

In [6]:
labels = 'Média < 5.0', '5.0 <= Média < 7.0', 'Média >= 7.0'
sizes = [len(red_edb1), len(green_edb1), len(blue_edb1)]

# Load data from dataframes into figure object
fig = {
    'data': [
        {
            'labels': labels,
            'values': sizes,
            'marker': {'colors': ['red', 'green', 'blue']},
            'type': 'pie',
            'name': 'notas',
            'domain': {'x': [.20, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        
    ],
    'layout':{
        'title': 'Média Final em EDB I dos alunos que tiveram média final em ITP acima de 7',
        'annotations':[
                {
                    'font': {'size': 20},
                    'showarrow': False,
                    'text': 'Situação',
                    'x': 0.6,
                    'y': 0.5
                }
        ]
    }
}

Nesta seção utilizamos a média em EDB1 dos alunos que foram aprovados em ITP com média $\geq 7,0$, para avaliar se os alunos que foram bem em ITP também foram bem em EBD1. No gráfico apresentado, o eixo X representa as médias em EDB1 (variando assim de 0 à 10) e o eixo Y representa as médias em ITP (variando assim de 7 à 10). Sendo assim, um ponto neste gráfico é um par ordenado $(média\_edb1, média\_itp)$ que refere-se à um aluno que cursou ITP, foi aprovado com média $\geq 7.0$ e pagou a disciplina de EDB1 no semestre seguinte. Esta análise é apresentada de duas formas (através de gráfico de dispersão e gráfico de setores) e é útil para avaliar se os alunos com bom desempenho na disciplina anterior continuavam a apresentar bom desempenho na disciplina seguinte. Nos dois gráficos os alunos estão agrupados de acordo com as cores vermelho ($média\_edb1 \lt 5,0$), verde ($5,0 \leq média\_edb1 \lt 7,0$) e azul ($média\_edb1 \gt 7,0$).

O gráfico de dispersão nos mostra em detalhes o desempenho em EDB1 daqueles que foram aprovados em ITP. A partir da dispersão dos pontos percebemos que os alunos que foram aprovados com média $\geq 8,5$ em ITP tiveram um desempenho melhor em EDB1, inclusive com notas melhores, do que aqueles com $7,0 \leq média\_itp \lt 8,5$. Além disso, todos os alunos que foram aprovados com média 10,0 em ITP conseguiram ser aprovados em EDB1.

Percebe-se também que a região do gráfico onde $7,0 \leq média\_itp \lt 8,5$ conta com um alto número de reprovados, mas, em geral, quem vai bem em ITP tende a passar em EDB1, e quem vai muito bem em ITP tem chances maiores ainda de ir bem em EDB1.

O gráfico de setores resume esses resultados. Dos alunos que foram aprovados em ITP com $média \geq 7,0$, cerca de $59,5\%$ conseguiu a aprovação em EDB1, sendo que $34\%$ daqueles conseguiram manter a $média \geq 7,0$. Todavia, percebe-se também uma elevada quantidade de reprovados ($40,4\%$).

A partir disso concluímos que é importante e útil aproveitar a disciplina de ITP para conseguir aproveitar a disciplina seguinte, que é EDB1.

In [7]:
output_notebook()
show(p)

Loading BokehJS ...

In [8]:
# Plot pie charts
py.iplot(fig)

## Análise 2 - Situação dos alunos que foram bem em EDBI

In [9]:
edb1_notas = {}
edb2_notas = {}

def retrieve_edb2_info(aid):
    tmp2 = pd.read_sql_query('''
            SELECT Aluno_ID, [Média Final]
            FROM Grade
            WHERE Aluno_ID = ''' + str(aid) + '''
                AND Disciplina = "IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II"
            LIMIT 1;''', engine
    )
    if tmp2.empty != True:
        try:
            media_edb2 = float(tmp2['Média Final'])
            return media_edb2
        except TypeError:
            return 0.0
    return None

# Execute query and store records in DataFrame: df
data_edb1_edb2 = pd.read_sql_query('''SELECT Aluno_ID, [Média Final]
                       FROM Grade
                       WHERE [Média Final] >= 7.0
                           AND Disciplina = "IMD0029 - ESTRUTURA DE DADOS BÁSICAS I"''',
                       engine)

# print the number of unique students
#print(len(df['Aluno_ID'].unique()))
for aid, med in zip(data_edb1_edb2['Aluno_ID'], data_edb1_edb2['Média Final']):
    edb2 = retrieve_edb2_info(aid)
    if edb2 != None:
        edb1_notas[int(aid)] = float(med)
        edb2_notas[int(aid)] = float(edb2)

In [10]:
red_edb1 = []
red_edb2 = []
green_edb1 = []
green_edb2 = []
blue_edb1 = []
blue_edb2 = []

for x in sorted(edb1_notas.keys()):
    if edb2_notas[int(x)] < 5.0:
        red_edb1.append(edb1_notas[x])
        red_edb2.append(edb2_notas[int(x)])
    elif edb2_notas[int(x)] >= 5.0 and edb2_notas[int(x)] < 7.0:
        green_edb1.append(edb1_notas[x])
        green_edb2.append(edb2_notas[int(x)])
    elif edb2_notas[int(x)] >= 7.0:
        blue_edb1.append(edb1_notas[x])
        blue_edb2.append(edb2_notas[int(x)])

In [11]:
p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Média em EDB2', y_axis_label = 'Média em EDB I',
           title = 'Desempenho dos alunos que cursaram EDB1 e EDB2 em seguida')

p.circle(x = red_edb2, y = red_edb1, color = "red", size = 7)
p.circle(x = green_edb2, y = green_edb1, color = "green", size = 7)
p.circle(x = blue_edb2, y = blue_edb1, color = "blue", size = 7)

# Load data from dataframes into figure object
labels = 'Média < 5.0', '5.0 <= Média < 7.0', 'Média >= 7.0'
sizes = [len(red_edb2), len(green_edb2), len(blue_edb2)]
fig = {
    'data': [
        {
            'labels': labels,
            'values': sizes,
            'marker': {'colors': ['red', 'green', 'blue']},
            'type': 'pie',
            'name': 'notas',
            'domain': {'x': [.20, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        
    ],
    'layout':{
        'title': 'Média Final em EDB II dos alunos que tiveram média final em EDB I acima de 7',
        'annotations':[
                {
                    'font': {'size': 20},
                    'showarrow': False,
                    'text': 'Situação',
                    'x': 0.6,
                    'y': 0.5
                }
        ]
    }
}

Nesta seção utilizamos a média em EDB2 dos alunos que foram aprovados em EDBI com média $\geq 7,0$, para avaliar se os alunos que foram bem em EDBI também foram bem em EBD2. No gráfico apresentado, o eixo X representa as médias em EDB2 (variando assim de 0 à 10) e o eixo Y representa as médias em EDBI (variando assim de 7 à 10). Sendo assim, um ponto neste gráfico é um par ordenado $(média\_edb2, média\_edb1)$ que refere-se à um aluno que cursou EDBI, foi aprovado com média $\geq 7.0$ e pagou a disciplina de EDB2 no semestre seguinte. Esta análise é apresentada de duas formas (através de gráfico de dispersão e gráfico de setores) e é útil para avaliar se os alunos com bom desempenho na disciplina anterior continuavam a apresentar bom desempenho na disciplina seguinte. Nos dois gráficos os alunos estão agrupados de acordo com as cores vermelho ($média\_edb2 \lt 5,0$), verde ($5,0 \leq média\_edb2 \lt 7,0$) e azul ($média\_edb2 \gt 7,0$).

O gráfico de dispersão nos mostra em detalhes o desempenho em EDB2 daqueles que foram aprovados em EDBI. A partir da dispersão dos pontos percebemos que os alunos que foram aprovados com $média \geq 7,0$ em EDB1 tiveram um desempenho muito bom em EDB2, com um total de aproximadamente $50\%$ da turma. Nota-se também que mesmo alunos que não tiraram 10,0 em EDB1 alcançaram a nota máxima em EDB2.

Podemos ver também, que o número de reprovados corresponde a apenas $12.9\%$ do total de alunos, e cerca de $37\%$ dos alunos com média $5,0 \leq média\_edb1 \lt 7,0$. Logo, $87\%$ dos que vão bem em EDB1, conseguem ser aprovados em EDB2.

O gráfico de setores resume esses resultados de cada uma das categorias. Em geral, os índices de aprovação aqui são melhores do que na Análise anterior.

A partir disso concluímos que é importante e útil aproveitar a disciplina de EDB1 para conseguir aproveitar a disciplina seguinte, que é EDB2.

In [12]:
output_notebook()
show(p)

Loading BokehJS ...

In [13]:
# Plot pie charts
py.iplot(fig)

## Análise 3 - Situação dos alunos que foram bem em PTP

In [14]:
ptp_notas = {}
lp1_notas = {}

def retrieve_lp1_info(aid):
    tmp = pd.read_sql_query('''
            SELECT Aluno_ID, [Média Final]
            FROM Grade
            WHERE Aluno_ID = ''' + str(aid) + '''
                AND Disciplina = "IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I"
            LIMIT 1;''', engine
    )
    if tmp.empty != True:
        try:
            media_lp1 = float(tmp['Média Final'])
            return media_lp1
        except TypeError:
            return 0.0
    return None

ptp_best = pd.read_sql_query('''SELECT Aluno_ID, [Média Final]
                       FROM Grade
                       WHERE [Média Final] >= 7.0
                           AND Disciplina = "IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO"''',
                       engine)

for aid, med in zip(ptp_best['Aluno_ID'], ptp_best['Média Final']):
    lp1 = retrieve_lp1_info(aid)
    if lp1 != None:
        ptp_notas[int(aid)] = float(med)
        lp1_notas[int(aid)] = float(lp1)

In [15]:
red_ptp = []
red_lp1 = []
green_ptp = []
green_lp1 = []
blue_ptp = []
blue_lp1 = []

for x in sorted(ptp_notas.keys()):
    if lp1_notas[int(x)] < 5.0:
        red_ptp.append(ptp_notas[x])
        red_lp1.append(lp1_notas[int(x)])
    elif lp1_notas[int(x)] >= 5.0 and lp1_notas[int(x)] < 7.0:
        green_ptp.append(ptp_notas[x])
        green_lp1.append(lp1_notas[int(x)])
    elif lp1_notas[int(x)] >= 7.0:
        blue_ptp.append(ptp_notas[x])
        blue_lp1.append(lp1_notas[int(x)])

p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Média em LP1', y_axis_label = 'Média em PTP',
           title = 'Desempenho dos alunos que cursaram PTP e LP1 em seguida')

p.circle(x = red_lp1, y = red_ptp, color = "red", size = 7)
p.circle(x = green_lp1, y = green_ptp, color = "green", size = 7)
p.circle(x = blue_lp1, y = blue_ptp, color = "blue", size = 7)

GlyphRenderer(id='65fe0ebd-3cf2-4168-a40e-f87ca1fc2f51', ...)

In [16]:
labels = 'Média < 5.0', '5.0 <= Média < 7.0', 'Média >= 7.0'
sizes = [len(red_lp1), len(green_lp1), len(blue_lp1)]

# Load data from dataframes into figure object
fig = {
    'data': [
        {
            'labels': labels,
            'values': sizes,
            'marker': {'colors': ['red', 'green', 'blue']},
            'type': 'pie',
            'name': 'notas',
            'domain': {'x': [.20, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        
    ],
    'layout':{
        'title': 'Média Final em LP I dos alunos que tiveram média final em PTP acima de 7',
        'annotations':[
                {
                    'font': {'size': 20},
                    'showarrow': False,
                    'text': 'Situação',
                    'x': 0.6,
                    'y': 0.5
                }
        ]
    }
}

Nesta seção utilizamos a média em LP1 dos alunos que foram aprovados em PTP com média $\geq 7,0$, para avaliar se os alunos que foram bem em PTP também foram bem em LP1. No gráfico apresentado, o eixo X representa as médias em LP1 (variando assim de 0 à 10) e o eixo Y representa as médias em PTP (variando assim de 7 à 10). Sendo assim, um ponto neste gráfico é um par ordenado $(média\_lp1, média\_ptp)$ que refere-se à um aluno que cursou PTP, foi aprovado com média $\geq 7.0$ e pagou a disciplina de LP1 no semestre seguinte. Esta análise é apresentada de duas formas (através de gráfico de dispersão e gráfico de setores) e é útil para avaliar se os alunos com bom desempenho na disciplina anterior continuavam a apresentar bom desempenho na disciplina seguinte. Nos dois gráficos os alunos estão agrupados de acordo com as cores vermelho ($média\_lp1 \lt 5,0$), verde ($5,0 \leq média\_lp1 \lt 7,0$) e azul ($média\_lp1 \gt 7,0$).

De forma similar à Análise 1, a partir da dispersão dos pontos percebemos que os alunos que foram aprovados com média $\geq 8,0$ em PTP tiveram um desempenho melhor em LP1 do que aqueles com $7,0 \leq média\_itp \lt 8,0$. Além disso, todos os alunos que foram aprovados com média 10,0 em PTP conseguiram ser aprovados em LP1, sendo que a maioria manteve o bom desempenho.

Percebe-se também que a região do gráfico onde $7,0 \leq média\_itp \lt 8,0$ conta com um alto número de reprovados, mas, em geral, quem vai bem em PTP tende a passar em LP1, e quem vai muito bem em PTP tem chances maiores ainda de ir bem em LP1.

O gráfico de setores resume esses resultados. Diferentemente da Análise 1, dos alunos que foram aprovados em PTP com $média \geq 7,0$, cerca de $57,9\%$ conseguiu a aprovação em LP1, sendo que $32,5\%$ daqueles conseguiram manter a $média \geq 7,0$. Percebe-se também uma quantidade um pouco maior de reprovados ($42\%$).

A partir disso concluímos que é importante e útil aproveitar a disciplina de PTP para conseguir aproveitar a disciplina seguinte, que é LP1.

In [17]:
output_notebook()
show(p)

Loading BokehJS ...

In [18]:
# Plot pie charts
py.iplot(fig)

## Análise 4 - Situação dos alunos que foram bem em LP1

In [19]:
lp1_notas = {}
lp2_notas = {}

def retrieve_lp2_info(aid):
    tmp = pd.read_sql_query('''
            SELECT Aluno_ID, [Média Final]
            FROM Grade
            WHERE Aluno_ID = ''' + str(aid) + '''
                AND Disciplina = "IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II"
            LIMIT 1;''', engine
    )
    if tmp.empty != True:
        try:
            media_lp2 = float(tmp['Média Final'])
            return media_lp2
        except TypeError:
            return 0.0
    return None

lp1_best = pd.read_sql_query('''SELECT Aluno_ID, [Média Final]
                       FROM Grade
                       WHERE [Média Final] >= 7.0
                           AND Disciplina = "IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I"''',
                       engine)

for aid, med in zip(lp1_best['Aluno_ID'], lp1_best['Média Final']):
    lp2 = retrieve_lp2_info(aid)
    if lp2 != None:
        lp1_notas[int(aid)] = float(med)
        lp2_notas[int(aid)] = float(lp2)

In [20]:
red_lp1 = []
red_lp2 = []
green_lp1 = []
green_lp2 = []
blue_lp1 = []
blue_lp2 = []

for x in sorted(lp1_notas.keys()):
    if lp2_notas[int(x)] < 5.0:
        red_lp1.append(lp1_notas[x])
        red_lp2.append(lp2_notas[int(x)])
    elif lp2_notas[int(x)] >= 5.0 and lp2_notas[int(x)] < 7.0:
        green_lp1.append(lp1_notas[x])
        green_lp2.append(lp2_notas[int(x)])
    elif lp2_notas[int(x)] >= 7.0:
        blue_lp1.append(lp1_notas[x])
        blue_lp2.append(lp2_notas[int(x)])

p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Média em LP2', y_axis_label = 'Média em LP1',
           title = 'Desempenho dos alunos que cursaram LP1 e LP2 em seguida')

p.circle(x = red_lp2, y = red_lp1, color = "red", size = 7)
p.circle(x = green_lp2, y = green_lp1, color = "green", size = 7)
p.circle(x = blue_lp2, y = blue_lp1, color = "blue", size = 7)

GlyphRenderer(id='dd19b6c6-38a8-403a-adc8-371cee09c3ba', ...)

In [21]:
labels = 'Média < 5.0', '5.0 <= Média < 7.0', 'Média >= 7.0'
sizes = [len(red_lp2), len(green_lp2), len(blue_lp2)]

# Load data from dataframes into figure object
fig = {
    'data': [
        {
            'labels': labels,
            'values': sizes,
            'marker': {'colors': ['red', 'green', 'blue']},
            'type': 'pie',
            'name': 'notas',
            'domain': {'x': [.20, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        
    ],
    'layout':{
        'title': 'Média Final em LP II dos alunos que tiveram média final em LP I acima de 7',
        'annotations':[
                {
                    'font': {'size': 20},
                    'showarrow': False,
                    'text': 'Situação',
                    'x': 0.6,
                    'y': 0.5
                }
        ]
    }
}

Nesta seção utilizamos a média em LP2 dos alunos que foram aprovados em LP1 com média $\geq 7,0$, para avaliar se os alunos que foram bem em LP1 também foram bem em LP2. No gráfico apresentado, o eixo X representa as médias em LP2 (variando assim de 0 à 10) e o eixo Y representa as médias em LP1 (variando assim de 7 à 10). Sendo assim, um ponto neste gráfico é um par ordenado $(média\_lp2, média\_lp1)$ que refere-se à um aluno que cursou LP1, foi aprovado com média $\geq 7.0$ e pagou a disciplina de LP2 no semestre seguinte. Esta análise é apresentada de duas formas (através de gráfico de dispersão e gráfico de setores) e é útil para avaliar se os alunos com bom desempenho na disciplina anterior continuavam a apresentar bom desempenho na disciplina seguinte. Nos dois gráficos os alunos estão agrupados de acordo com as cores vermelho ($média\_lp2 \lt 5,0$), verde ($5,0 \leq média\_lp2 \lt 7,0$) e azul ($média\_lp2 \gt 7,0$).

A partir da dispersão dos pontos percebemos que, em geral, os alunos foram aprovados com média $\geq 7,0$ em LP1 tiveram um bom desempenho LP2. Nota-se também que mesmo alunos que não tiraram 10,0 em LP1 se aproximaram da nota máxima em LP2.

O gráfico de setores apresenta o bom aproveitamento, onde $91,7\%$ dos bons alunos de LP1 também conseguiram aprovação em LP2, sendo que $63,9\%$ conseguiu manter o bom desempenho. A taxa de reprovação é menor do que em EDB2, tendo apenas $8,25\%$.

Como nas Análises anteriores, percebe-se a importância de aproveitar a disciplina de LP1 para conseguir aproveitar a disciplina seguinte, que é LP2.

In [22]:
output_notebook()
show(p)

Loading BokehJS ...

In [23]:
# Plot pie charts
py.iplot(fig)

## Análise 5 - Evolução das disciplinas teóricas e práticas

In [24]:
teoricas_data = pd.read_sql_query('''SELECT Período, Disciplina, AVG([Média Final]) as Media
                       FROM Grade
                       WHERE Disciplina = "IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO" OR
                           Disciplina = "IMD0029 - ESTRUTURA DE DADOS BÁSICAS I" OR
                           Disciplina = "IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II"
                       GROUP BY Período''',
                       engine)

praticas_data = pd.read_sql_query('''SELECT Período, Disciplina, AVG([Média Final]) as Media
                       FROM Grade
                       WHERE Disciplina = "IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO" OR
                           Disciplina = "IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I" OR
                           Disciplina = "IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II"
                       GROUP BY Período''',
                       engine)

In [25]:
source = ColumnDataSource(data = {
        'periodos': ['2014.1', '2014.2', '2015.1', '2015.2', '2016.1', '2016.2'],
        'medias_teo': [x for x in teoricas_data['Media']],
        'medias_pra': [x for x in praticas_data['Media']]
    }
)

hover = HoverTool(
    tooltips=[
        ("Período", "@periodos"),
        ("Téoricas", "@medias_teo"),
        ("Práticas", "@medias_pra")
    ]
)

p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Períodos', y_axis_label = 'Médias',
           x_range = source.data['periodos'], title = 'Teóricas vs. Práticas', tools=[hover])

p.line(x = 'periodos', y = 'medias_teo', color = Spectral6[0], line_width = 4, source = source, legend = 'Teóricas')
p.circle(x = 'periodos', y = 'medias_teo', fill_color = "white", size = 7, source = source)

p.line(x = 'periodos', y = 'medias_pra', color = Spectral6[1], line_width = 4, source = source, legend = 'Práticas')
p.circle(x = 'periodos', y = 'medias_pra', fill_color = "white", size = 7, source = source)

p.legend.location = "top_left"

Nesta seção utilizamos as médias das disciplinas teóricas e das disciplinas práticas, por período, para avaliar em qual tipo os alunos tendem a ter um melhor desempenho. Foram consideradas disciplinas teóricas, por serem majoritariamente teóricas: ITP, EDB1 e EDB2. Foram consideradas disciplinas práticas, por serem majoritariamente práticas: PTP, LP1 e LP2. As disciplinas EDB1, EDB2, LP1 e LP2 só constam nos registros a partir do período 2014.2.

Primeiramente, para esta análise foi plotado um gráfico de linhas que contém as médias das disciplinas teóricas e das disciplinas práticas. Entre os períodos 2014.1 e 2015.1 os alunos apresentavam um desempenho melhor nas disciplinas práticas, sendo que a diferença de média entre elas foi diminuindo, até que em 2015.2 as duas médias se aproximaram bastante, com uma diferença de 0,11. A partir de 2016.1 o cenário foi invertido e, aparentemente, os alunos passaram a se sair melhor nas disciplinas teóricas.

In [26]:
output_notebook()
show(p)

Loading BokehJS ...

In [27]:
def get_course_average(course):
    return pd.read_sql_query('''SELECT Período, Disciplina, AVG([Média Final]) as Media
                       FROM Grade
                       WHERE Disciplina = "''' + course + '''"
                       GROUP BY Período''',
                       engine)

itp_data = get_course_average("IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO")
edb1_data = get_course_average("IMD0029 - ESTRUTURA DE DADOS BÁSICAS I")
edb2_data = get_course_average("IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II")
ptp_data = get_course_average("IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO")
lp1_data = get_course_average("IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I")
lp2_data = get_course_average("IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II")

In [28]:
source1 = ColumnDataSource(data = {
        'periodos': ['2014.1', '2014.2', '2015.1', '2015.2', '2016.1', '2016.2'],
        'itp': [x for x in itp_data['Media']],
        'ptp': [x for x in ptp_data['Media']]
    }
)

source2 = ColumnDataSource(data = {
        'periodos': ['2014.2', '2015.1', '2015.2', '2016.1', '2016.2'],
        'edb1': [x for x in edb1_data['Media']],
        'edb2': [x for x in edb2_data['Media']],
        'lp1': [x for x in lp1_data['Media']],
        'lp2': [x for x in lp2_data['Media']]
    }
)

p = figure(plot_width = 900, plot_height = 500, x_axis_label = 'Períodos', y_axis_label = 'Médias',
           x_range = source.data['periodos'], title = 'Evolução das disciplinas de programação')

p.line(x = 'periodos', y = 'itp', color = Spectral6[0], line_width = 4, source = source1, legend = 'ITP')
p.circle(x = 'periodos', y = 'itp', fill_color = "white", size = 7, source = source1)

p.line(x = 'periodos', y = 'ptp', color = Spectral6[1], line_width = 4, source = source1, legend = 'PTP')
p.circle(x = 'periodos', y = 'ptp', fill_color = "white", size = 7, source = source1)

p.line(x = 'periodos', y = 'edb1', color = Spectral6[2], line_width = 4, source = source2, legend = 'EDB1')
p.circle(x = 'periodos', y = 'edb1', fill_color = "white", size = 7, source = source2)

p.line(x = 'periodos', y = 'lp1', color = Spectral6[3], line_width = 4, source = source2, legend = 'LP1')
p.circle(x = 'periodos', y = 'lp1', fill_color = "white", size = 7, source = source2)

p.line(x = 'periodos', y = 'edb2', color = Spectral6[4], line_width = 4, source = source2, legend = 'EDB2')
p.circle(x = 'periodos', y = 'edb2', fill_color = "white", size = 7, source = source2)

p.line(x = 'periodos', y = 'lp2', color = Spectral6[5], line_width = 4, source = source2, legend = 'LP2')
p.circle(x = 'periodos', y = 'lp2', fill_color = "white", size = 7, source = source2)

p.legend.location = "top_left"

A fim de melhor analisar o ocorrido foi plotado outro gráfico de linhas contendo todas as seis disciplinas de programação. A partir dele percebe-se que o primeiro gráfico vem sendo fortemente afetado pelo desempenho nas disciplinas de ITP e PTP, e, por motivo desconhecido, as disciplinas práticas vêm se tornando mais desafiadoras aos alunos. Isso é refletido no primeiro gráfico. Até a disciplina de LP2, a qual possui a maior média, vem perdendo espaço para a sua co-requisito EDB2.

É notável também uma maior instabilidade de desempenho nas disciplinas ITP, PTP, EDB1 e EDB2. As disciplinas introdutórias ITP/PTP deveriam ser priorizadas para que o aluno possa aproveitar melhor as disciplinas seguintes e é preocupante vê-las com baixo desempenho em alguns semestres (como em 2016.1).

In [29]:
output_notebook()
show(p)

Loading BokehJS ...

## Análise 6 - Número de Reprovados x Disciplina em todos os anos.

In [30]:
data_rep = pd.read_sql_query('''SELECT Disciplina, Período, Count(Aluno_ID) as count
                       FROM Grade
                       WHERE Situaçăo = "REPROVADO"
                       GROUP BY Disciplina, Período''', engine)
                       
data_rep.head()

,Disciplina,Período,count
0,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2014.1,13
1,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2014.2,18
2,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2015.1,6
3,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2015.2,29
4,IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO,2016.1,20


Nesta seção utilizamos os dados do número de reprovados em cada uma das disciplinas disponíveis, em todos os anos.

Como podemos ver abaixo,as disciplinas Introdução às Técnicas de Programação (ITP) e Práticas de Técnicas de Programação (PTP), Estruturas de dados Básicas I (EDBI) e Laboratório de Programação I (LPI), talvez por serem disciplinas iniciais e primeiros contatos dos alunos com programação, possuem um número de reprovações bem elevado em relação a EDBII e LP2, acumulando aproximadamente 100 reprovações no total.

In [31]:
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,wheel_zoom, pan'
Bar_Reprovados = Bar(data_rep, 'Disciplina', values='count',color = 'Disciplina', xlabel="Disciplina",  
         ylabel = "Nº de Reprovados", title="Disciplinas pelo Nº de Reprovados", 
         legend = False,tools=[TOOLS])

output_notebook()
show(Bar_Reprovados)

Loading BokehJS ...

## Análise 7 - Disciplinas x Notas em cada ano

In [32]:
data_2 = pd.read_sql_query("select Período, Disciplina, [Média Final] from Grade",engine)
dataYear2014_nota1 = data_2[data_2["Período"] == 2014.1]
dataYear2015_nota1 = data_2[data_2["Período"] == 2015.1]
dataYear2016_nota1 = data_2[data_2["Período"] == 2016.1]
dataYear2014_nota2 = data_2[data_2["Período"] == 2014.2]
dataYear2015_nota2 = data_2[data_2["Período"] == 2015.2]
dataYear2016_nota2 = data_2[data_2["Período"] == 2016.2]


NotasYear2014_1 = BoxPlot(dataYear2014_nota1, values='Média Final', label='Disciplina', color = 'Disciplina', legend = False, 
             title = "BoxPlot das disciplinas pelas notas, Ano 2014.1", tools = [TOOLS])

NotasYear2015_1 = BoxPlot(dataYear2015_nota1, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2015.1", tools = [TOOLS])

NotasYear2016_1 = BoxPlot(dataYear2016_nota1, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2016.1", tools = [TOOLS])

NotasYear2014_2 = BoxPlot(dataYear2014_nota2, values='Média Final', label='Disciplina', color = 'Disciplina', legend = False, 
             title = "BoxPlot das disciplinas pelas notas, Ano 2014", tools = [TOOLS])

NotasYear2015_2 = BoxPlot(dataYear2015_nota2, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2015.1", tools = [TOOLS])

NotasYear2016_2 = BoxPlot(dataYear2016_nota2, values='Média Final', label='Disciplina', color = 'Disciplina',legend = False,
            title = "BoxPlot das disciplinas pelas notas, Ano 2016.1", tools = [TOOLS])

Nesta seção utilizamos os dados de cada uma das disciplinas pelas notas, em cada ano em que foram ofertadas.

Analisando as disciplinas, a grande maioria delas se mantém no mesmo nível ao longo dos períodos. Um destaque para essa análise é ofato de que as medianas estão sempre próximas de 5 e 6, o que podemos concluir que em algumas disciplinas, como EDBI e LPI, a maioria dos alunos ou está sendo reprovada ou passando com dificudade. Também podemos notar a disciplina de LPI que no ano de 2016 teve sua mediana menor que os anos anteriores, na qual no ano de 2016, cerca de $50\%$ dos alunos ficaram com média bem próximos a 5 e mediana da turma aproximadamente 3,2.

In [33]:
row1 = [NotasYear2014_1,NotasYear2014_2]

# Create a list containing plots p3 and p4: row2
row2 = [NotasYear2015_1,NotasYear2015_2]

row3 = [NotasYear2016_1,NotasYear2016_2]

# Create a gridplot using row1 and row2: layout
layout = gridplot([row1,row2,row3],plot_width=480, plot_height=700)

output_notebook()


show(layout)

Loading BokehJS ...

## Análise 8 - Disciplinas x Notas em todos os anos.

Nesta seção utilizamos os dados de cada uma das disciplinas pelas notas, em cada ano em que foram ofertadas.

Analisando as disciplinas, como dito anteriormente, no geral, as notas estão com medianas entre 5 e 6. As disciplinas iniciais ITP, PTP, EDBI e LP1, possuem uma distribuição menos concentra em relação a EDII e LPII. Destque LP2 com mais de $50\%$ da turma acima de 5.2 e com as melhores notas, levando em consideração as disciplinas de programação.

In [34]:
BoxPlot_Nota = BoxPlot(data_2, values='Média Final', label='Disciplina', legend = False, color = 'Disciplina', tools = [TOOLS])

output_notebook()
show(BoxPlot_Nota)

Loading BokehJS ...

## Análise 9 - Disciplina x Quantidade de vezes que alunos se matricularam

In [35]:
# Filter by qtd_matriculas and disciplina ITP
d0 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0012.0 - INTRODUÇĂO ŔS TÉCNICAS DE PROGRAMAÇĂO"''',engine)
d0['count'] = 0
d0 = pd.DataFrame(d0.groupby('Aluno_ID')['count'].count()).reset_index()
d0 = d0.rename(columns = {'count':'qtd_matriculas'})
d0['count'] = 0
d0 = pd.DataFrame(d0.groupby('qtd_matriculas')['count'].count()).reset_index()
d0 = d0.loc[d0['qtd_matriculas'] != 6]
d0 = d0.loc[d0['qtd_matriculas'] != 5]

# Filter by qtd_matriculas and disciplina PTP
d1 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0012.1 - PRÁTICAS DE TÉCNICAS DE PROGRAMAÇĂO"''',engine)
d1['count'] = 0
d1 = pd.DataFrame(d1.groupby('Aluno_ID')['count'].count()).reset_index()
d1 = d1.rename(columns = {'count':'qtd_matriculas'})
d1['count'] = 0
d1 = pd.DataFrame(d1.groupby('qtd_matriculas')['count'].count()).reset_index()
d1 = d1.loc[d1['qtd_matriculas'] != 6]
d1 = d1.loc[d1['qtd_matriculas'] != 5]

# Filter by qtd_matriculas and disciplina EDB I
d2 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0029 - ESTRUTURA DE DADOS BÁSICAS I"''',engine)
d2['count'] = 0
d2 = pd.DataFrame(d2.groupby('Aluno_ID')['count'].count()).reset_index()
d2 = d2.rename(columns = {'count':'qtd_matriculas'})
d2['count'] = 0
d2 = pd.DataFrame(d2.groupby('qtd_matriculas')['count'].count()).reset_index()
d2 = d2.loc[d2['qtd_matriculas'] != 6]
d2 = d2.loc[d2['qtd_matriculas'] != 5]

# Filter by qtd_matriculas and disciplina LP I
d3 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0030 - LINGUAGEM DE PROGRAMAÇĂO I"''',engine)
d3['count'] = 0
d3 = pd.DataFrame(d3.groupby('Aluno_ID')['count'].count()).reset_index()
d3 = d3.rename(columns = {'count':'qtd_matriculas'})
d3['count'] = 0
d3 = pd.DataFrame(d3.groupby('qtd_matriculas')['count'].count()).reset_index()
d3 = d3.loc[d3['qtd_matriculas'] != 6]
d3 = d3.loc[d3['qtd_matriculas'] != 5]

# Filter by qtd_matriculas and disciplina EDB II
d4 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0039 - ESTRUTURAS DE DADOS BÁSICAS II"''',engine)
d4['count'] = 0
d4 = pd.DataFrame(d4.groupby('Aluno_ID')['count'].count()).reset_index()
d4 = d4.rename(columns = {'count':'qtd_matriculas'})
d4['count'] = 0
d4 = pd.DataFrame(d4.groupby('qtd_matriculas')['count'].count()).reset_index()
d4 = d4.loc[d4['qtd_matriculas'] != 6]
d4 = d4.loc[d4['qtd_matriculas'] != 5]

# Filter by qtd_matriculas and disciplina LP II
d5 = pd.read_sql_query('''select Aluno_ID from Grade where Disciplina = "IMD0040 - LINGUAGEM DE PROGRAMAÇĂO II"''',engine)
d5['count'] = 0
d5 = pd.DataFrame(d5.groupby('Aluno_ID')['count'].count()).reset_index()
d5 = d5.rename(columns = {'count':'qtd_matriculas'})
d5['count'] = 0
d5 = pd.DataFrame(d5.groupby('qtd_matriculas')['count'].count()).reset_index()
d5 = d5.loc[d5['qtd_matriculas'] != 6]
d5 = d5.loc[d5['qtd_matriculas'] != 5]


In [36]:
# Load data from dataframes into figure object
fig1 = {
    'data': [
        {
            'labels': d2['qtd_matriculas'],
            'values': d2['count'],
            'type': 'pie',
            'name': 'EDB I',
            'domain': {'x': [0, .48], 'y': [0, 0.49]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        {
            'labels': d3['qtd_matriculas'],
            'values': d3['count'],
            'type': 'pie',
            'name': 'LP I',
            'domain': {'x': [.52, 1], 'y': [0, 0.49]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        {
            'labels': d0['qtd_matriculas'],
            'values': d0['count'],
            'type': 'pie',
            'name': 'ITP',
            'domain': {'x': [0, .48], 'y': [0.51, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        {
            'labels': d1['qtd_matriculas'],
            'values': d1['count'],
            'type': 'pie',
            'name':'PTP',
            'domain': {'x': [.52, 1], 'y': [0.51, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        }
    ],
    'layout':{
        'title': 'Disciplina pela Quantidade de vezes que alunos se matricularam',
        'annotations':[
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'ITP',
                    'x': 0.22,
                    'y': 0.8
                },
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'PTP',
                    'x': 0.78,
                    'y': 0.8
                },
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'EDB I',
                    'x': 0.22,
                    'y': 0.21
                },
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'LP I',
                    'x': 0.78,
                    'y': 0.21
                }
        ]
    }
}

In [37]:
fig2 = {
    'data': [
        {
            'labels': d4['qtd_matriculas'],
            'values': d4['count'],
            'type': 'pie',
            'name': 'Disciplina 4',
            'domain': {'x': [0, .48], 'y': [0.51, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        },
        {
            'labels': d5['qtd_matriculas'],
            'values': d5['count'],
            'type': 'pie',
            'name':'Disciplina 5',
            'domain': {'x': [.52, 1], 'y': [0.51, 1]},
            'hoverinfo':'label+percent+name+value',
            'textposition': 'outside',
            'hole': '.4'
        }
    ],
    'layout':{
        'title': 'Disciplina pela Quantidade de vezes que alunos se matricularam',
        'annotations':[
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'EDB II',
                    'x': 0.21,
                    'y': 0.8
                },
                {
                    'font': {'size': 15},
                    'showarrow': False,
                    'text': 'LP II',
                    'x': 0.78,
                    'y': 0.8
                }
        ]
    }
}

De acordo com o regulamento da UFRN, é previsto o cancelamento do programa se houver desempenho acadêmico insuficiente por parte do aluno. Ainda de acordo com o regulamento, **Seção III (15.16.3. DO DESEMPENHO ACADÊMICO INSUFICIENTE), Art. 327 - I**:

***"Caracteriza-se o desempenho acadêmico insuficiente quando, em um período letivo regular no qual o programa não está suspenso, ocorre uma ou mais das seguintes situações: Insucesso (trancamento e/ou reprovação) pela quarta vez ou mais, consecutiva ou não, em um mesmo componente curricular obrigatório ou seus equivalentes;"***


Com o objetivo de verificar o andamento de cada disciplina e assim analisar quais têm causado mais dificuldades aos alunos, inclusive abrindo possibilidades de cancelamento de seus programas, levantamos os dados que mostram a quantidade de vezes que os alunos se matricularam nas disciplinas no decorrer do tempo (intervalo de 2014 à 2016).

In [38]:
# Plot pie charts
py.iplot(fig1)

In [39]:
py.iplot(fig2)